# Project 2

In [28]:
import torch
import math
torch.set_grad_enabled(False)
torch.manual_seed(123)

In [2]:
class Module(object):
    def __init__(self):
        self.parameters = []
        
    def forward(self, *input):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return self.parameters

In [3]:
class Linear(Module):
    def __init__(self, in_features, out_features, bias=True):
        init_range = 1. / math.sqrt(in_features)
        self.weights = torch.Tensor(in_features, out_features).uniform_(-init_range, init_range)
        self.grad_w = torch.zeros((in_features, out_features))
        self.parameters = [(self.weights, self.grad_w)]
        if bias:
            self.bias = torch.Tensor(out_features).uniform_(-init_range, init_range)
            self.grad_b = torch.zeros(out_features)
            self.parameters.append((self.bias, self.grad_b))
        else:
            self.bias = None
        
    def forward(self, input_):
        self.input = input_
        if self.bias != None:
            return torch.addmm(self.bias, input_, self.weights)
        else:
            return input_.matmul(self.weights)
        
    def backward(self, grad_output):
        self.grad_w += self.input.t().matmul(grad_output)
        grad_input = grad_output.matmul(self.weights.t())
        if self.bias != None:
            self.grad_b += grad_output.sum(dim=0)
        return grad_input

In [4]:
class ReLU(Module):
    def forward(self, input_):
        self.input = input_
        return torch.relu(input_)
    
    def backward(self, grad_output):
        return torch.mul((self.input > 0).int(), grad_output)

In [5]:
class Tanh(Module):
    def forward(self, input_):
        self.input = input_
        return torch.tanh(input_)
    
    def backward(self, grad_output):
        return torch.tanh(self.input).pow(2).mul(-1).add(1).mul(grad_output)

In [6]:
class Sequential(Module):
    def __init__(self, *args):
        self.layers = list(args)
        self.parameters = []
        for module in args:
            self.parameters += module.parameters
    
    def forward(self, input_):
        x = input_
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, loss_grad):
        y = loss_grad
        for layer in reversed(self.layers):
            y = layer.backward(y)
        return

In [7]:
class MSELoss(Module):
    def __init__(self):
        pass
    
    def __call__(self, input_, target):
        return self.forward(input_, target)
    
    def forward(self, input_, target):
        if target.dim() == 1:
            target = target.view(target.size(0), 1)
        return (input_ - target).pow(2).sum().item()
    
    def backward(self, input_, target):
        if target.dim() == 1:
            target = target.view(target.size(0), 1)
        
        return (input_ - target).mul(2)

In [8]:
sample_size = 1000

In [9]:
def generate_set(size):
    input_ = torch.Tensor(size, 2).uniform_(0, 1)
    target_single = input_.sub(0.5).pow(2).sum(axis=1).sub(1 / (math.pi * 2)).sign().view(-1, 1)
    target = torch.cat((target_single.mul(-1), target_single), 1).add(1).div(2)
    return input_, target

train_input, train_target = generate_set(sample_size)
test_input, test_target = generate_set(sample_size)

In [10]:
class Model:
    def __init__(self, layers):
        self.layers = layers
        self.parameters = []
        for layer in layers:
            self.parameters += layer.parameters
    
    def __call__(self, input_):
        return self.forward(input_)
    
    def zero_grad(self):
        for w, dw in self.parameters:
            dw.zero_()
    
    def forward(self, input_):
        x = input_
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, loss_grad):
        y = loss_grad
        for layer in reversed(self.layers):
            y = layer.backward(y)
        return

In [11]:
class SGD:
    def __init__(self, parameters, learning_rate):
        self.parameters = parameters
        self.learning_rate = learning_rate
        
    def step(self):
        for w, dw in self.parameters:
            w.sub_(self.learning_rate * dw)

In [20]:
layers = [Linear(2, 25), ReLU(), Linear(25, 25), ReLU(), Linear(25, 25), ReLU(), Linear(25, 2), Tanh()]
model = Model(layers)
loss = MSELoss()

In [24]:
def train_model(model, train_input, train_target, batch_size=100, n_epochs=100, loss=MSELoss(), learning_rate=0.001, print_loss=True):
    sample_size = train_input.size(0)
    print(sample_size)
    sgd = SGD(model.parameters, learning_rate)
    for epoch in range(n_epochs):
        cumulative_loss = 0
        for n_start in range(0, sample_size, batch_size):
            model.zero_grad()
            output = model(train_input[n_start : n_start + batch_size])
            cumulative_loss += loss(output, train_target[n_start : n_start + batch_size])
            loss_grad = loss.backward(output, train_target[n_start : n_start + batch_size])
            model.backward(loss_grad)
            sgd.step()
        if print_loss:
            print("Epoch: %i" % epoch)
            print("Loss: %f" % cumulative_loss)

In [25]:
train_model(model, train_input, train_target)

1000
Epoch: 0
Loss: 87.831915
Epoch: 1
Loss: 89.499357
Epoch: 2
Loss: 87.446156
Epoch: 3
Loss: 87.051387
Epoch: 4
Loss: 85.301146
Epoch: 5
Loss: 85.314953
Epoch: 6
Loss: 85.250834
Epoch: 7
Loss: 84.456578
Epoch: 8
Loss: 83.065378
Epoch: 9
Loss: 82.181355
Epoch: 10
Loss: 79.679480
Epoch: 11
Loss: 81.547590
Epoch: 12
Loss: 81.215204
Epoch: 13
Loss: 79.794634
Epoch: 14
Loss: 80.366846
Epoch: 15
Loss: 79.551367
Epoch: 16
Loss: 78.953803
Epoch: 17
Loss: 79.431170
Epoch: 18
Loss: 76.218543
Epoch: 19
Loss: 77.240423
Epoch: 20
Loss: 75.720995
Epoch: 21
Loss: 77.332279
Epoch: 22
Loss: 76.078060
Epoch: 23
Loss: 73.418399
Epoch: 24
Loss: 75.921631
Epoch: 25
Loss: 73.486027
Epoch: 26
Loss: 71.474735
Epoch: 27
Loss: 71.287388
Epoch: 28
Loss: 71.462109
Epoch: 29
Loss: 73.669683
Epoch: 30
Loss: 65.930800
Epoch: 31
Loss: 71.172777
Epoch: 32
Loss: 68.472447
Epoch: 33
Loss: 68.141115
Epoch: 34
Loss: 67.151998
Epoch: 35
Loss: 67.491766
Epoch: 36
Loss: 66.161403
Epoch: 37
Loss: 66.764684
Epoch: 38
Loss: 6

In [67]:
def accuracy(true_target, predicted):
    return true_target.argmax(dim=1).sub(predicted.argmax(dim=1)).eq(0).float().mean().item()